In [1]:
!pip install pandas
!pip install torch
!pip install nltk
!pip install tqdm
!pip install seaborn
!pip install numpy
!pip install sklearn

In [3]:
import nltk
nltk.download('punkt')

# Скачиваем данные

In [4]:
!wget https://raw.githubusercontent.com/semensorokin/DLforNLP_course_material/master/Homework2/answers_subsample.csv

In [ ]:
# если ругается на то, что нет wget
# !apt-get install wget

In [6]:
!ls -l

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('answers_subsample.csv')

In [9]:
data

In [10]:
data.category.value_counts() * 100 / data.shape[0]

# Предобученные эмбеддинги
[Источник](https://fasttext.cc/docs/en/crawl-vectors.html)  
Вы можете взять любые word2vec подобные эмббединги. Если вы хотите использовать elmo, bert, etc сначала попробуйте с word2vec подобными эмббедингами, а потом можете перейти к более сложным моделям.  
Ниже мы сначала скачиваем, а потом распоковываем эмбеддинги.

In [11]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

In [12]:
!ls -l

In [13]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

In [14]:
# потом можете добавить свою предобработку

def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [15]:
word2freq = {}
lengths = []

for text in tqdm(data.text):
    
    words = process_text(text)
    
    lengths.append(len(words))
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

In [16]:
import seaborn as sns
from matplotlib import pyplot as plt

In [17]:
plt.figure(figsize=(10, 6))
plt.title('Распределение длин слов в текстах')
plt.xlabel('Длина предложения')
plt.ylabel('Доля')
sns.distplot(lengths)

In [18]:
upper_threshold = 32
lower_threshold = 3

correct_percent = len([sent_len for sent_len in lengths 
                       if sent_len <= upper_threshold and sent_len >= lower_threshold]) * 100 / len(lengths)

'{:.2f} % наших текстов входят в промежуток от {} до {} слов'.format(correct_percent, lower_threshold, upper_threshold)

In [19]:
len(word2freq)

In [20]:
'{} слов, которые встречались 3 и менее раз'.format(len([word for word in word2freq if word2freq[word] <= 3]))

# Читаем файл с эмбеддингами
### Этот файл с 300 числами для 2 000 000 слов и он может не влезть в память
Поэтому прочитаем только те слова, которые мы знаем

In [21]:
import numpy as np

In [22]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('cc.ru.300.vec')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)

while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()

    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

In [23]:
len(word2index)

In [24]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

# Потеря 2.5 % слов в датасете
Эта ситуация не то, чтобы сильно плохая, в учебных целях нормально, к тому же в среднем они редко встречаются. Вы можете поиграть с предобработкой.

In [25]:
import torch

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [26]:
x = torch.rand(128, 64, 1024)

In [27]:
lstm = torch.nn.LSTM(1024, 512, batch_first=True)

In [28]:
%%timeit

pred = lstm(x)

# А что GPU?

In [29]:
print('Доступна ли видеокарта:', torch.cuda.is_available())
print('Если недоступна, поменяйте runtime, если в колабе')

In [30]:
# универсальных способ задать device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# если доступна gpu, то давайте ее использовать, но в этом задании должны использовать

In [31]:
# перенесли x на gpu
x_gpu = x.to(device)

In [32]:
# зададим lstm на gpu
lstm_gpu = torch.nn.LSTM(1024, 512, batch_first=True)
lstm_gpu = lstm_gpu.to(device)

In [33]:
%%timeit

pred = lstm_gpu(x_gpu)

# У меня на 1070 TI скорость уменьшилась с 381мс до 41мс, то есть в 9.29 раз

In [34]:
# если у нас модель на гпу, а то, что мы туда подаем нет, то работать не будет
# справедлива и обратная ситуация

# выскочит ошибка
# посмотрите на нее, возможно, вы еще встретитесь
# pred = lstm_gpu(x)

# Важные и не очень интуитивные моменты про LSTM и CNN в торче

По умолчанию LSTM принимает данные с такой размерностью:
```python
(seq_len, batch, input_size)
```
Сделано это с целью оптимизации на более низком уровне.  
Мы оперируем такими объектами:
```python
(batch, seq_len, input_size)
```
Чтобы LSTM у нас заработала правильно, мы можем либо передать параметр ```batch_first=True``` во время инициализации слоя,
либо транспонировать (поменять) первую и вторую размерность у нашего x перед подачей в слой.  
[Подробнее про LSTM](https://pytorch.org/docs/stable/nn.html#lstm)

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [35]:
# первый способ
lstm = torch.nn.LSTM(1024, 512, batch_first=True)

pred, mem = lstm(x)

In [36]:
pred.shape

In [37]:
lstm = torch.nn.LSTM(1024, 512)

# меняем размерность batch и seq_len местами
x_transposed = x.transpose(0, 1)
pred_transposed, mem = lstm(x_transposed)

In [38]:
# у нас все еще осталась размерность (seq_len, batch, input_size)
pred_transposed.shape

In [39]:
# просто транспонируем еще раз
pred = pred_transposed.transpose(0, 1)
pred.shape

## Conv1d & MaxPool1d
Примерно такая же ситуация происходит со сверточными слоями и пулингами.  
1d реализация как раз для текстов, в ней матрица-фильтр ходит только по одной размерности.  
[Подробнее про CNN](https://pytorch.org/docs/stable/nn.html#conv1d)  
[Подробнее про пулинг](https://pytorch.org/docs/stable/nn.html#maxpool1d)  
Ожидается такая размерность:
```python
(batch, input_size, seq_len)
```
Мы все еще хоти подавать такую размерность:
```python
(batch, seq_len, input_size)
```
В случае со свертками и пулингами у нас есть вариант только транспонировать x перед подачей и транспонировать полученный результат. Обратите внимание, что транспонируем мы первую и вторую размерность (индексация с нуля).

In [40]:
x.shape

- 128 - размер батча
- 64 - количество слов
- 1024 - эмбеддинг слова

In [41]:
# in_channels - размер входных эмбеддингов
# out_channels - количество/какой размер эмбеддингов мы хотим получить
# kernel_size - размер окна/н-граммы
cnn = torch.nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3)

In [42]:
# выпадет ошибка, посмотрите какая
# pred = cnn(x)

In [43]:
x_transposed = x.transpose(1, 2)
x_transposed.shape
# перевели в (batch, input_size, seq_len)

In [44]:
pred_transposed = cnn(x_transposed)
pred_transposed.shape
# осталась разрмерность (batch, output_size, seq_len)

In [45]:
# переведем обратно в (batch, seq_len, input_size)
pred = pred_transposed.transpose(1, 2)
pred.shape

# Подготовим данные в DataLoader

In [46]:
from torch.utils.data import Dataset, DataLoader

In [47]:
'UNK' in word2index

In [48]:
data.head()

# Замапим категории в индексы

In [49]:
cat_mapper = {cat: n for n, cat in enumerate(data.category.unique())}

In [50]:
cat_mapper

In [51]:
data.category = data.category.map(cat_mapper)

# Читалка данных

## Что происходит ниже
1. Мы задаем x_data, y_data (таргеты), word2index (маппер из слова в индекс слова), sequence_length (максимальная длина последовательности, если больше, ограничить ею), pad_token (токен паддинга и задаем его индекс pad_index).
1. Загружаем данные:
    1. Проходимся по датасету
    1. Предобрабатываем каждый текст в датасете
    1. Индексируем его
    1. Паддим до нужной длины
1. Когда нам нужно достать пример из датасета мы берем индексированный ```x``` и соответствующий этому индексу ```y```, наш ```x``` также паддим (или ограничиваем длину) и переводим в ```torch.Tensor(x).long()```. Для ```y``` этого делать не потребуется, в dataloader'е таргеты преобразуются в тензор сами.


In [52]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=32, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
        
        words = wordpunct_tokenize(text.lower())
        #words = re.findall('[a-яА-ЯеЁ]+', text.lower())
        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            
            words = self.process_text(text)
            
            indexed_words = self.indexing(words)
            
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        return [self.word2index[word] for word in tokenized_text if word in self.word2index]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        if len(sequence)< self.sequence_length:
            add_pad = self.sequence_length - len(sequence)
            return sequence+[self.pad_index]*add_pad
        else:
            return sequence[:self.sequence_length]
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [54]:
x_train, x_validation, y_train, y_validation = train_test_split(data.text, data.category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=64)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=64)

In [55]:
for x, y in train_loader:
    break

In [56]:
x

In [57]:
y

# Обучить нейронку

In [58]:
from torch import nn, optim

In [59]:
from math import sqrt

class model_with_att(torch.nn.Module):
    def __init__(self, matrix_w, n, hidden_size=256, conv_out=128, lin_out=256): #n - количество категорий
        super().__init__()
        self.n = n
        self.embedding_dim = matrix_w.shape[1]

        self.hidden_size = hidden_size
        self.conv_out = conv_out
        self.lin_out = lin_out

        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w))

        # задайте лстм, можно 2 уровня, лучше бидирекциональный, в доке торча есть инофрмация как это сделать в одну строчку
        self.LSTM = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_size,
                            num_layers=2, batch_first=True, bidirectional=True, dropout=0.1)

        # три линейных преобразования, размерность совпадает с выходом из лстм (если БИлстм то надо умножить ее на 2)
        self.q_proj = nn.Linear(in_features=self.hidden_size*2, out_features=self.hidden_size)
        self.k_proj = nn.Linear(in_features=self.hidden_size*2, out_features=self.hidden_size)
        self.v_proj = nn.Linear(in_features=self.hidden_size*2, out_features=self.hidden_size)

        self.att_soft = torch.nn.Softmax(dim = 2)
        
        # три конволюционных фильтра с разными ядрами (3,4,5) чтобы были всякие нграммы ловить
        self.cnn_3gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=3, padding='same')
        self.cnn_4gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=4, padding='same')
        self.cnn_5gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=5, padding='same')

        # сверху накидываем два полносвязных слоя для классификации
        self.linear_1 = nn.Linear(in_features=self.conv_out*3, out_features=self.lin_out)
        self.relu = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(in_features=lin_out, out_features=n) 

        
    def forward(self, x):
        #примените эмбеддинги
        x_emb = self.emb_layer(x)
        # транспонируйте тензор для лстм как было описано выше
        # нет нужды, т.к. batch_first=True
        # применим лстм, не забываем что на выходе у него много всяких последовательностей, нам нужна только эта
        x, _ = self.LSTM(x_emb)
        # транспонируйте обратно

        x_q = self.q_proj(x) #применим линейные преобразования для селф-эттеншена
        x_k = self.k_proj(x)
        x_v = self.v_proj(x)
        x_k_t = x_k.transpose(2,1)
        # посмотрите в презентацию и перемножьте нужные тензора изспольуя функцию bmm из торча, перед этим одну из матриц обзательно транспонируйте
        # результат обязательно поделите на корень из последней размерности (то есть на размер эмбеддинга из предыдущего слоя)
        att_scores = torch.bmm(x_q, x_k_t) / sqrt(self.embedding_dim)

        att_dist = self.att_soft(att_scores) # накидываем софтмакс
        # тут тоже что то с чем то нужно перемножить :)
        attention_vectors = torch.bmm(att_dist, x_v)

        x_att = attention_vectors.transpose(2,1) #транспонируем для конволюционных фильтров

        x_cnn3 = self.cnn_3gr(x_att)
        x_cnn4 = self.cnn_4gr(x_att)
        x_cnn5 = self.cnn_5gr(x_att)

        frst, _ =  x_cnn3.max(dim= -1,) # cделаем макс пуллинг
        sc, _ = x_cnn4.max(dim= -1,)
        thr, _ = x_cnn5.max(dim= -1,)
      
        x_cat = torch.cat((frst, sc, thr), dim=-1) # а теперь объединим результаты
      
        # пару полносвязных слоев с релу для классификации
        x = self.linear_1(x_cat)
        x = self.relu(x)    
        x = self.linear_2(x)
    
        return x

In [60]:
n_classes = data.category.unique().shape[0]

In [61]:
model = model_with_att(vectors, n_classes)

In [63]:
model #если сделать batch_first=True, то можно не транспонировать батчи

In [64]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [65]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

model = model.to(device)
criterion = criterion.to(device)

In [66]:
epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), position=0, leave=True,
                        desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = model(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)

        with torch.no_grad():

            pred = model(x)

            pred = pred.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Если вы запускаете много раз колаб окна и ткдм начинает беситься, можно запустить окно ниже, ткдм обновится и все снова станет хорошо

In [113]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)

# Оценка
1. Добрались сюда - очень хорошо - получилась такая же точность или около того - 7 баллов.
2. Поставили эксперименты и повысили точность относительно своей и не ниже F1 test - 0.841 - 8 баллов.
3. Запустили бертовую тетрадку и разобрались. Получился сравнимый результат - 10 баллов 

# Улучшение
Хочу попробовать добавить больше параметров и слоёв!

In [67]:
from IPython import display
display.Image('https://habrastorage.org/files/040/6ca/59e/0406ca59e7c243e1bffae413d1d40947.png',
              width = 400)

In [123]:
class ImprovedModel(torch.nn.Module):
    def __init__(self, matrix_w, n, hidden_size=300, conv_out=200, lin_out=300): #n - количество категорий
        super().__init__()
        self.n = n
        self.embedding_dim = matrix_w.shape[1]

        self.hidden_size = hidden_size
        self.conv_out = conv_out
        self.lin_out = lin_out

        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w), freeze=True)

        # задайте лстм, можно 2 уровня, лучше бидирекциональный, в доке торча есть информация как это сделать в одну строчку
        self.biLSTM = nn.LSTM(input_size=self.embedding_dim, hidden_size=int(self.hidden_size*1.25),
                            num_layers=2, batch_first=True, bidirectional=True, dropout=0.4)
        # поверх би- хочу обычную
        self.LSTM = nn.LSTM(input_size=int(self.hidden_size*1.25), hidden_size=self.hidden_size,
                            num_layers=1, batch_first=True)

        # три линейных преобразования, размерность совпадает с выходом из лстм (если БИлстм то надо умножить ее на 2)
        self.q_proj = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.k_proj = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.v_proj = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)

        self.att_soft = torch.nn.Softmax(dim = 2)
        
        # три конволюционных фильтра с разными ядрами (3,4,5) чтобы были всякие нграммы ловить
        # а потом ещё триграммы по сумме 3,4,5
        self.cnn_3gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=3, padding='same')
        self.cnn_4gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=4, padding='same')
        self.cnn_5gr = nn.Conv1d(in_channels=self.hidden_size, out_channels=self.conv_out,
                                 kernel_size=5, padding='same')
        self.cnn_cat = nn.Conv1d(in_channels=self.conv_out*3, out_channels=self.conv_out,
                                 kernel_size=3, padding='same')

        # сверху накидываем два полносвязных слоя для классификации
        self.linear_1 = nn.Linear(in_features=self.conv_out, out_features=self.lin_out)
        self.relu = torch.nn.PReLU()
        self.linear_2 = torch.nn.Linear(in_features=lin_out, out_features=n) 

        
    def forward(self, x):
        #примените эмбеддинги
        x_emb = self.emb_layer(x)
        # транспонируйте тензор для лстм как было описано выше
        # нет нужды, т.к. batch_first=True
        # хочу два слоя biLSTM, а потом ещё два обычной
        _, (h, _) = self.biLSTM(x_emb)
        h = h.transpose(0,1)
        x, _ = self.LSTM(h)
        # транспонируйте обратно

        x_q = self.q_proj(x) #применим линейные преобразования для селф-эттеншена
        x_k = self.k_proj(x)
        x_v = self.v_proj(x)
        x_k_t = x_k.transpose(2,1)
        # посмотрите в презентацию и перемножьте нужные тензора изспольуя функцию bmm из торча, перед этим одну из матриц обзательно транспонируйте
        # результат обязательно поделите на корень из последней размерности (то есть на размер эмбеддинга из предыдущего слоя)
        att_scores = torch.bmm(x_q, x_k_t) / sqrt(self.embedding_dim)

        att_dist = self.att_soft(att_scores) # накидываем софтмакс
        # тут тоже что то с чем то нужно перемножить :)
        attention_vectors = torch.bmm(att_dist, x_v)

        x_att = attention_vectors.transpose(2,1) #транспонируем для конволюционных фильтров

        x_cnn3 = self.cnn_3gr(x_att)
        x_cnn4 = self.cnn_4gr(x_att)
        x_cnn5 = self.cnn_5gr(x_att)
        x_cat = torch.cat((x_cnn3, x_cnn4, x_cnn5), dim=1) # а теперь объединим результаты
        conv_cat = self.cnn_cat(x_cat)
        
        pooled, _ =  conv_cat.max(dim= -1,) # cделаем макс пуллинг
      
        # пару полносвязных слоев с релу для классификации
        x = self.linear_1(pooled)
        x = self.relu(x)    
        x = self.linear_2(x)
    
        return x

In [124]:
improved_model = ImprovedModel(vectors, n_classes)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=improved_model.parameters())

improved_model = improved_model.to(device)
criterion = criterion.to(device)

In [128]:
improved_model

In [125]:
epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), position=0, leave=True,
                        desc='Epoch {}'.format(n_epoch + 1))
    
    improved_model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = improved_model(x)
        
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    improved_model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)

        with torch.no_grad():

            pred = improved_model(x)

            pred = pred.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Успех! На базовой модели в последних двух эпохах f1 был 0.844, на этой модели также на последних двух 0.846. Что поменялось:
* поверх двух слоёв biLSTM добавила один слой обычной LSTM
* у biLSTM увеличила дропаут с 0.1 до 0.4
* сложила свёртки 3-, 4- и 5-тиграмм и поверх этого сделала ещё один свёрточный слой с окном 3, только затем пулинг
* увеличила количество параметров: hidden size и lin out с 256 до 300, conv_out с 128 до 200
* заменила ReLU на PReLU